In [1]:
SEED = 11

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from IPython.display import display, Math

In [3]:
import torch.nn as nn
import torch.optim as optim
# import torch.nn.init as init
# from torchvision import datasets
# from torchvision import transforms
# from torch.utils.data import DataLoader

In [4]:
n_hidden = 35
lr = 0.01
epochs = 1000

In [7]:
my_string = 'hello pytorch. how long can a rnn cell remember? i am a boy and you are a girl. can you make a pancake for mommy?'
chars = 'abcdefghijklmnopqrstuvwxyz ?!.,:;01'
char_list = [i for i in chars]
n_letters = len(char_list)

In [8]:
def str2onehot(string):
    start = np.zeros(shape = n_letters, dtype = int)
    end = np.zeros(shape = n_letters, dtype = int)
    start[-2] = 1
    end[-1] = 1
    for c in string:
        idx = char_list.index(c)
        zero = np.zeros(shape = n_letters, dtype = int)
        zero[idx] = 1
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

def onehot2word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [9]:
class my_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(my_RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.in2hid = nn.Linear(input_size, hidden_size)
        self.hid2hid = nn.Linear(hidden_size, hidden_size)
        self.hid2out = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
        
    def forward(self, input, hidden):
        hidden = self.act_fn(self.in2hid(input)+self.hid2hid(hidden))
        
#         hidden = self.act_fn(self.hid2hid(hidden)+self.hid2hid(hidden))
        output = self.hid2out(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    

    

In [10]:
rnn = my_RNN(n_letters, n_hidden, n_letters)

In [11]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = lr)

In [12]:
one_hot = torch.from_numpy(str2onehot(my_string)).type_as(torch.FloatTensor())

In [13]:
for ii in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()
    
    ## go all sequence and sum all loss
    for jj in range(one_hot.size()[0]-1):
        input_ = one_hot[jj:jj+1, :]
        target = one_hot[jj+1]
        
        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        
        total_loss += loss
        input_ = output
    
    # update weight after calculating all loss through sequence
    total_loss.backward()
    optimizer.step()
    
    if ii%10 == 0:
        print(total_loss)
    
    

tensor(5.7975, grad_fn=<AddBackward0>)
tensor(2.5315, grad_fn=<AddBackward0>)
tensor(1.9824, grad_fn=<AddBackward0>)
tensor(1.6172, grad_fn=<AddBackward0>)
tensor(1.4545, grad_fn=<AddBackward0>)
tensor(1.2237, grad_fn=<AddBackward0>)
tensor(1.0580, grad_fn=<AddBackward0>)
tensor(0.9933, grad_fn=<AddBackward0>)
tensor(0.8684, grad_fn=<AddBackward0>)
tensor(0.8172, grad_fn=<AddBackward0>)
tensor(0.7504, grad_fn=<AddBackward0>)
tensor(0.6984, grad_fn=<AddBackward0>)
tensor(0.6648, grad_fn=<AddBackward0>)
tensor(0.8776, grad_fn=<AddBackward0>)
tensor(0.6723, grad_fn=<AddBackward0>)
tensor(0.6207, grad_fn=<AddBackward0>)
tensor(0.5896, grad_fn=<AddBackward0>)
tensor(0.6440, grad_fn=<AddBackward0>)
tensor(0.5631, grad_fn=<AddBackward0>)
tensor(0.5353, grad_fn=<AddBackward0>)
tensor(0.5923, grad_fn=<AddBackward0>)
tensor(0.5205, grad_fn=<AddBackward0>)
tensor(0.4978, grad_fn=<AddBackward0>)
tensor(0.5484, grad_fn=<AddBackward0>)
tensor(0.4908, grad_fn=<AddBackward0>)
tensor(0.4671, grad_fn=<A

In [14]:
total_loss

tensor(0.2760, grad_fn=<AddBackward0>)

In [15]:
hidden # weight of 2nd hidden layer 

tensor([[-0.1255, -0.0404, -0.6356,  0.1626, -0.0112, -0.4381,  0.3585,  0.5888,
          0.0530, -0.8257,  0.1955,  0.3604, -0.2566, -0.5012,  0.4825, -0.5124,
          0.3779, -0.3167, -0.2582, -0.2034,  0.4278,  0.2537,  0.3895, -0.7635,
         -0.6118,  0.6108,  0.2875,  0.2417,  0.6414, -0.6434,  0.0534,  0.0189,
          0.2960,  0.1863,  0.1595]], grad_fn=<TanhBackward>)

In [16]:
output.shape

torch.Size([1, 35])

In [17]:
start = torch.zeros(1, n_letters)
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ''
    for ii in range(len(my_string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot2word(output.data)
        input_ = output
        
print(output_string)

hell    oanaa    a  a                                                                                            


In [18]:
rnn.forward(input_, hidden)

(tensor([[ 1.7619e-01, -1.4109e-02,  3.0419e-02,  8.0557e-03,  1.6432e-02,
           1.0327e-02,  3.6577e-02,  3.3711e-03,  3.3934e-02,  3.7498e-19,
          -2.8715e-03,  9.1544e-03,  1.1991e-01,  7.9978e-02,  9.2735e-02,
           2.9714e-02,  1.2301e-11,  4.3104e-02,  7.9051e-09,  1.5728e-03,
           1.6422e-02,  4.5057e-16,  1.7457e-03, -1.0289e-07,  6.1112e-02,
           5.2734e-08,  2.3803e-01,  8.5765e-03,  3.7316e-19, -3.1280e-03,
           7.7459e-09, -6.0652e-19,  3.3331e-15,  7.1422e-20,  8.6302e-04]],
        grad_fn=<AddmmBackward>),
 tensor([[-0.0507, -0.0083, -0.0213, -0.3745, -0.1168, -0.0159,  0.1221,  0.0798,
           0.0404, -0.1461,  0.0151,  0.0551, -0.0605,  0.0892, -0.0876,  0.0042,
           0.1800, -0.0039,  0.1578,  0.1172, -0.1664,  0.0234, -0.0425, -0.1243,
          -0.0022, -0.0126, -0.2337, -0.0215,  0.0131, -0.0886, -0.0136, -0.0254,
          -0.0415, -0.1307, -0.2510]], grad_fn=<TanhBackward>))

아주 간단히 구현한 것이고, 학습한 문자열도 길지 않아 엄청 좋은 결과를 얻기는 힘들다.
그저 torch로 rnn을 어떻게 구현하는지에 대해서 초점을 맞추자